In [ ]:
%pylab inline
plt.style.use("bmh")
plt.rcParams["figure.figsize"] = (12,4)

In [ ]:
import pathlib

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
DATA_DIR = pathlib.Path("data")

# Outline

The model is loosely based on [Enhancing the Locality and Breaking the Memory Bottleneck of Transformer on Time Series Forecasting
](https://arxiv.org/abs/1907.00235). We do not implement "learnable" positional encoding, allowing convolutions to do the job. Outputs are distribution parameters, similar to DeepAR.

# Loading data

In [ ]:
df = pd.read_csv("data/AEP_hourly.csv", parse_dates=["Datetime"], index_col="Datetime")

## Fix timestamps

In [ ]:
df.index.is_monotonic, df.index.is_unique

In [ ]:
df = df.sort_index()
df

### New index

In [ ]:
new_idx = pd.date_range("2004-10-01 01:00:00", "2018-08-03 00:00:00", freq="1H")

In [ ]:
dfi = df[~df.index.duplicated(keep='first')].reindex(new_idx)

In [ ]:
dfi.index.is_monotonic, dfi.index.is_unique, dfi.index.freq

### Missing values

In [ ]:
dfi.ffill(inplace=True)

# Various model blocks

In [ ]:
# PyTorch imports
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset

# PyTorch Lightning imports
import pytorch_lightning as pl

# Causal convolution

In [ ]:
class CausalConv1D(pl.LightningModule):
    """Causal convolution for transformer model."""
   
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: int,
                 padding_mode: str = 'zeros'):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding_mode = padding_mode
        self.padding_requirement = kernel_size - 1

        self.layer = nn.Conv1d(self.in_channels,
                               self.out_channels,
                               self.kernel_size,
                               padding=self.padding_requirement,
                               padding_mode=self.padding_mode)

    def forward(self, x):
        x_out = self.layer(x)
        return x_out[:, :, :-self.padding_requirement]

In [ ]:
test_input = torch.Tensor(np.random.randn(1, 4, 100))

causal_conv = CausalConv1D(4, 8, 3)

In [ ]:
test_output = causal_conv(test_input)

In [ ]:
test_output.shape

In [ ]:
causal_conv.layer.weight.shape

# Positional encoding

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 dim: int,
                 max_len: int = 168,
                 dropout: float = 0.1):
        super(PositionalEncoding, self).__init__()

        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(dim, max_len)
        position = torch.arange(0, max_len, dtype=torch.float)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-math.log(10000.0) / dim)).unsqueeze(1)
        pe[0::2, :] = torch.sin(position * div_term)
        pe[1::2, :] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :, :x.size(-1)]
        return self.dropout(x)

In [ ]:
pos_enc = PositionalEncoding(8, 100)

In [ ]:
pos_enc.pe.shape

In [ ]:
plt.imshow(pos_enc.pe.numpy()[0])

In [ ]:
pos_enc(test_output).shape

In [ ]:
plt.imshow(pos_enc(test_output).detach().numpy()[0])

In [ ]:
plt.imshow(test_output.detach().numpy()[0])

# `TransformerEncoder` layer

In [ ]:
tenc_layer = nn.TransformerEncoderLayer(8, 8, 16)
test_enc = tenc_layer(pos_enc(test_output).permute(2, 0, 1))

In [ ]:
test_enc.shape

In [ ]:
tenc_layer.self_attn

# Dataset and DataModule

In [ ]:
class ElectricityDataset(Dataset):
    """Dataset which samples the data from hourly electricity data."""

    def __init__(self, df, samples, hist_len=168, fct_len=24, col="AEP_MW"):
        self.hist_num = hist_len
        self.fct_num = fct_len
        self.hist_len = pd.Timedelta(hours=hist_len)
        self.fct_len = pd.Timedelta(hours=fct_len)
        self.offset = pd.Timedelta(hours=1)

        self.max_ts = df.index.max() - self.hist_len - self.fct_len + self.offset
        self.raw_data = df

        assert samples <= self.raw_data[:self.max_ts].shape[0]
        self.samples = samples
        self.col = col
        self.sample()

    def sample(self):
        """Sample individual series as needed."""
        
        self.sample_idx = (self
                           .raw_data[:self.max_ts]
                           .index
                           .to_series()
                           .sample(self.samples, replace=False)
                           .index)

    def __len__(self):
        return self.samples

    def __getitem__(self, idx):
        start_ts = self.sample_idx[idx]

        hs, he = start_ts, start_ts + self.hist_len - self.offset
        fs, fe = he + self.offset, he + self.fct_len

        hist_data = self.raw_data[hs:].iloc[:self.hist_num]
        fct_data = self.raw_data[fs:].iloc[:self.fct_num]

        return (torch.Tensor(hist_data[self.col].values).unsqueeze(0),
                torch.Tensor(fct_data[self.col].values).unsqueeze(0))


class ElectricityDataModule(pl.LightningDataModule):
    """DataModule for electricity data."""

    def __init__(self, df,
                 train_range=("2004", "2015"),
                 val_range=("2016","2017"),
                 test_range=("2018", None),
                 factor=0.5,
                 batch_size=64,
                 workers=3):

        super().__init__()
        self.raw_data = df
        self.train_range = train_range
        self.val_range = val_range
        self.test_range = test_range
        self.factor = factor
        self.batch_size = batch_size
        self.workers = workers

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            train_df = self.raw_data[slice(*self.train_range)]
            val_df = self.raw_data[slice(*self.val_range)]

            self.train_ds = ElectricityDataset(train_df,
                                               samples=int(self.factor * train_df.shape[0]))
            self.val_ds = ElectricityDataset(val_df,
                                             samples=int(self.factor * val_df.shape[0]))

        if stage == "test" or stage is None:
            test_df = self.raw_data[slice(*self.test_range)]
            self.test_ds = ElectricityDataset(test_df,
                                              samples=int(self.factor * test_df.shape[0]))

    
    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, num_workers=self.workers)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, num_workers=self.workers)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size, num_workers=self.workers)

In [ ]:
ds = ElectricityDataset(dfi, 10)

# Model

In [ ]:
class ElectricityLoadTransformer(pl.LightningModule):
    """ransformer model for electricity load forecasting."""
   
    def __init__(self,
                 hist_len: int = 168,
                 fct_len: int = 24,
                 kernels: int = 8,
                 kernel_size: int = 5,
                 pos_dropout: float = 0.1,
                 tr_dropout: float = 0.1,
                 heads: int = 8,
                 dim_feedforward: int = 32,
                 lr: float = 1e-3):
        super().__init__()

        self.hist_len = hist_len
        self.fct_len = fct_len
        self.kernels = kernels
        self.kernel_size = kernel_size
        self.pos_dropout = pos_dropout
        self.tr_dropout = tr_dropout
        self.heads = heads
        self.dim_feedforward = dim_feedforward
        self.lr = lr
        
        self.conv_encoder = CausalConv1D(1, self.kernels, self.kernel_size)
        self.pos_encoder = PositionalEncoding(self.kernels, self.hist_len+self.fct_len, self.pos_dropout)
        self.transformer_block = nn.TransformerEncoderLayer(d_model=self.kernels,
                                                            nhead=self.heads,
                                                            dim_feedforward=self.dim_feedforward,
                                                            dropout=self.tr_dropout)

        self.mu = nn.Linear(in_features=self.kernels, out_features=1)
        self.sigma_raw = nn.Linear(in_features=self.kernels, out_features=1)
        self.sigma = nn.Softplus()

        # Mask
        mask = torch.triu(torch.ones(self.hist_len + self.fct_len,
                                     self.hist_len + self.fct_len), diagonal=1) == 1
        self.register_buffer("mask", mask)
        

    def forward(self, x):
        
        conv_enc = self.conv_encoder(x)
        pos_enc = self.pos_encoder(conv_enc)
        local_mask = self.mask[:pos_enc.size(-1), :pos_enc.size(-1)]
        
        transformed = self.transformer_block(pos_enc.permute(2, 0, 1), src_mask=local_mask)

        mu = self.mu(transformed)
        sigma = self.sigma(self.sigma_raw(transformed))
        return transformed, mu, sigma

    def step(self, batch, batch_idx, tag="train"):
        combined_input = torch.cat(batch, dim=-1)

        # Pushing through the network
        out, mu, sigma = self(combined_input[:, :, :-1])
        loss = self.loss(mu, sigma, combined_input[:, :, 1:].permute(2, 0, 1))
        self.log(f'{tag}_logprob', loss, prog_bar=True)
        return loss
    
    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)

    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, tag="val")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def sample(self, x, samples):
        # Handle single stream or multiple streams
        # Note the differences in indexing compared to DeepAR
        x = x.view(-1, 1, self.hist_len)

        # Initial pass - `mu(T+1)` and `sigma(T+1)` are ready
        out, mu, sigma = self(x)

        # Sample from the distribution
        gaussian = torch.distributions.normal.Normal(mu[-1, :, -1], sigma[-1, :, -1])
        initial_sample = gaussian.sample((samples,))

        all_samples = []

        # Iterating over samples
        for sample in range(samples):
            current_sample = initial_sample[sample].view(-1, 1, 1)
            step_in = torch.cat([x, current_sample], dim=-1)
            
            # Iterating over time steps
            for step in range(self.fct_len - 1):
                step_out, mu, sigma = self(step_in[:, :, step+1:])

                # Sampling the next step value
                gaussian = torch.distributions.normal.Normal(mu[-1, :, -1], sigma[-1, :, -1])
                current_sample = gaussian.sample((1,)).view(-1, 1, 1)

                # Input tensor for the next step
                step_in = torch.cat([step_in, current_sample], dim=-1)
            all_samples.append(step_in[:, :, -self.fct_len:].unsqueeze(-1))
        return torch.cat(all_samples, dim=-1)

    def loss(self, mu, sigma, y):
        # Distribution with generated `mu` and `sigma`
        gaussian = torch.distributions.normal.Normal(mu, sigma)

        # Log-likelihood
        L = gaussian.log_prob(y)

        return -torch.mean(L)

In [ ]:
model = ElectricityLoadTransformer()
model

In [ ]:
model_input = torch.Tensor(np.random.randn(2, 1, 168))
model_output, mu, sigma = model(model_input)

In [ ]:
model_output.shape, mu.shape, sigma.shape

In [ ]:
samples = model.sample(model_input, 3)

In [ ]:
samples.shape

# Training

In [ ]:
LIMH, LIML =26e3, 9e3

In [ ]:
dfs = (2 * dfi - LIML - LIMH) / (LIMH - LIML)

In [ ]:
ds = ElectricityDataModule(dfs, batch_size=32)
model = ElectricityLoadTransformer(kernels=64, heads=8, dim_feedforward=512)
trainer = pl.Trainer(max_epochs=10, progress_bar_refresh_rate=1)
trainer.fit(model, ds)

# Testing the model

In [ ]:
ds.setup("test")
dl = ds.test_dataloader()

hist, fct = next(iter(dl))

In [ ]:
for stream in range(10):
    sampled = model.sample(hist[[stream]], 32)
    sampled_mean = sampled.mean(dim=-1)
    sampled_qhigh = sampled.quantile(0.75, dim=-1)
    sampled_qlow = sampled.quantile(0.25, dim=-1)

    plt.figure(figsize=(12,4))

    plt.plot(hist[stream][0], label="historical data")
    plt.plot(np.arange(168, 192, 1), fct[stream].detach().numpy()[0], label="actual")
    plt.plot(np.arange(168, 192, 1), sampled_mean.detach().numpy()[0, 0],
             color="white",
             label="forecast mean")
    plt.fill_between(np.arange(168, 192, 1),
                     sampled_qlow.detach().numpy()[0, 0],
                     sampled_qhigh.detach().numpy()[0, 0],
                     label="forecast interval", color="forestgreen", alpha=0.6)

    plt.legend(loc=0)
    plt.tight_layout()
    plt.show()

# Inside the model

In [ ]:
plt.imshow(model.conv_encoder(hist[[0]]).detach().numpy()[0])

In [ ]:
plt.imshow(model.pos_encoder(model.conv_encoder(hist[[0]])).detach().numpy()[0])

In [ ]:
pos_enc = model.pos_encoder(model.conv_encoder(hist[[0]])).permute(2, 0, 1)
out, attn = model.transformer_block.self_attn(pos_enc, pos_enc, pos_enc, attn_mask=model.mask[:168, :168])

In [ ]:
out.shape, attn.shape

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(np.log(1+attn[0].detach().numpy()), vmax=0.15)